# Map Matching Pipeline for beautofuel
## Beware that:
### - code is not functional
### - this is experimental only

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
import geopy.distance
from influxdb import DataFrameClient

from lib.packages.eda_quality import manipulation as manipulate
from lib.packages.eda_quality import inspection as inspect
from lib.utils.constants import INFLUXDB_HOST, INFLUXDB_PORT, INFLUXDB_USER, INFLUXDB_PASSWORD, INFLUXDB_DB

## Data interpolation

In [ ]:
interpolated_tracks_df=None

# Interpolate every track
# TODO Interpolate only missing GPS coordinates (track cuts)
for track_id in track_ids:
    track_df = inspect.get_single_track(tracks_df, track_id)

    track_df = manipulate.interpolate(track_df)

    interpolated_tracks_df = pd.concat([interpolated_tracks_df, track_df])

tracks_df = interpolated_tracks_df

tracks_df.plot()

## Tracks map matching

In [26]:
%%time
import folium
import numpy as np
import osmnx as ox

from leuvenmapmatching.matcher.distance import DistanceMatcher
from leuvenmapmatching.map.inmem import InMemMap

map_matched_track_coords_dict = {}

tracks_count = len(track_ids)

print("Total number of tracks to analyze is {}\n".format(tracks_count))

for i in range(tracks_count):
    print("Analysing track {} with ID {}".format(i + 1, track_ids[i]))

    track_df = inspect.get_single_track(tracks_df, track_ids[i])

    # Calculate centroid coordinate
    # Get the latitude and longitude coordinates from the track and join them in an array
    lngs = list(track_df['geometry'].apply(lambda coord: coord.x))
    lats = list(track_df['geometry'].apply(lambda coord: coord.y))

    centroid_lng = sum(lngs)/len(lngs)
    centroid_lat = sum(lats)/len(lats)

    centroid = (centroid_lat, centroid_lng)

    print("Track centroid is located at {}".format(centroid))

    # Calculate interval times
    dt = np.zeros(len(track_df['time']))

    for j in range(1, len(track_df['time'])):
        dt[j] = get_interval_time(track_df['time'].iloc[j], track_df['time'].iloc[j-1])
    
    speed = np.array(track_df['GPS Speed.value'])

    distance_total = np.sum(speed / 3.6 * dt) / 1000

    dist = int(distance_total / 2 * 1000)

    print("Graph from point distance is {}".format(dist))

    graph = ox.graph_from_point(centroid, dist=dist, network_type='drive')

    # ox.plot_graph(graph)

    # OSM graph transformation for compatibility with LMM
    # Leuven Map Matching is using a different internal graph structure for the street data. 
    # Therefore, the OSMnx graph needs to be transformed to the InMemMap
    streetmap = InMemMap("enviroCar", use_latlon=True, use_rtree=True, index_edges=True)

    # add nodes
    nodes = list(graph.nodes)
    for node in nodes:
        lng = graph.nodes[node]['x']
        lat = graph.nodes[node]['y']
        streetmap.add_node(node, (lat, lng))

    # add edges
    edges = list(graph.edges)
    for edge in edges:
        node_a, node_b = edge[0], edge[1]
        streetmap.add_edge(node_a, node_b)
        
        # exclude bi-directional edges when street is oneway
        if not graph.edges[edge]['oneway']:
            streetmap.add_edge(node_b, node_a)
        
    streetmap.purge()

    # Map matching
    track_coords = [c for c in zip(lats, lngs)]

    # TODO Calculate programmatically
    max_dist = 300

    # Configure the mapmatcher. For increasing the performance it is important 
    # that max_dist (in meters) is not set too high
    matcher = DistanceMatcher(streetmap,
                            max_dist=max_dist, 
                            max_dist_init=50,
                            min_prob_norm=0.001,
                            non_emitting_length_factor=0.75,
                            obs_noise=50,
                            obs_noise_ne=75,
                            dist_noise=50,
                            non_emitting_edgeid=False)

    # Perform the mapmatching 
    try:
        edge_ids, last_idx = matcher.match(track_coords)
    except Exception:
        print("Map matching cannot be performed for this track\n")
        continue
    if not matcher.lattice_best:
        print("No matching track has been found\n")
        continue

    # get the coordinates of the "best" mapmatched route
    coords = [m.edge_m.pi[:] for m in matcher.lattice_best]

    map_matched_track_coords_dict[track_ids[i]] = coords

    # Visualize matched map
    # plot both raw and mapmatched track
    # m = folium.Map(location=[lat, lng], zoom_start=13)
    # folium.PolyLine(coords, color='blue').add_to(m)
    # folium.PolyLine(track_coords, color='red').add_to(m)
    # m

    print()

Total number of tracks to analyze is 17

Analysing track 1 with ID 6005b338886b71633ce138ed
Track centroid is located at (49.084432660036235, 20.18333248467393)
Graph from point distance is 9627


KeyboardInterrupt: 

## Tracks similiarity search

In [ ]:
print("There are {} tracks suitable for similarity search".format(len(map_matched_track_coords_dict)))

# for track_id in map_matched_track_coords_dict:
#     map_matched_track_coords = map_matched_track_coords_dict[track_id]

# print(map_matched_track_coords_dict['5fc3d8e605fa792e88632925'])
# print(map_matched_track_coords_dict['5fc3d8e605fa792e886329e7'])

# for coord in map_matched_track_coords_dict['5fc3d8e605fa792e88632925']:


